In [1]:
# This notebook is for training the ATPC data with a 3D CNN

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import sys
sys.path.append("../scripts")
from TrackReconstruction_functions import *

import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc


os.environ["OMP_NUM_THREADS"] = "16"
import MinkowskiEngine as ME

%matplotlib widget

In [2]:
# Check for CUDA (NVIDIA) or MPS (Apple)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

NVIDIA GeForce RTX 2060
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Using device: cuda


In [3]:
# Load in the signal metadata
nubb_meta = pd.read_hdf(f"/media/argon/HardDrive_8TB/Krishan/ATPC/trackreco/ATPC_0nubb/1bar/5percent/reco_filtered/ATPC_0nubb_1bar_5percent_filtered.h5", "MC/hits")
nubb_meta["Type"] = "0nubb"
nubb_meta["subType"] = "0nubb"
# display(nubb_meta) 

# ----------------------------------------------------------------------------------------------------

# Load in the background
Bkg_meta = pd.DataFrame()
Tl_meta = pd.DataFrame()
Bi_meta = pd.DataFrame()

Tl_meta = pd.read_hdf(f"/media/argon/HardDrive_8TB/Krishan/ATPC/trackreco/ATPC_Tl_ion/1bar/5percent/reco_filtered/ATPC_Tl_ion_1bar_5percent_filtered.h5", "MC/hits")
Tl_meta["subType"] = "Tl"
Bi_meta = pd.read_hdf(f"/media/argon/HardDrive_8TB/Krishan/ATPC/trackreco/ATPC_Bi_ion/1bar/5percent/reco_filtered/ATPC_Bi_ion_1bar_5percent_filtered.h5", "MC/hits")
Bi_meta["subType"] = "Bi"
single_meta = pd.read_hdf(f"/media/argon/HardDrive_8TB/Krishan/ATPC/trackreco/ATPC_single/1bar/5percent/reco_filtered/ATPC_single_1bar_5percent_filtered.h5", "MC/hits")
single_meta["subType"] = "single"

Bkg_meta = pd.concat([Tl_meta, Bi_meta, single_meta])

Bkg_meta["Type"] = "Bkg"


# display(Bkg_meta)

df = pd.concat([nubb_meta, Bkg_meta])
# df = df.drop(columns=['x_smear', 'y_smear', 'z_smear'])
df = df[["event_id", "x", "y", "z", "energy", "Type"]]
display(df)

,event_id,x,y,z,energy,Type
0,122,-1554.677368,-1489.785767,1508.124512,0.034792,0nubb
1,122,-1543.320435,-1489.691711,1512.508423,0.094080,0nubb
2,122,-1538.769165,-1480.243164,1513.530762,0.042178,0nubb
3,122,-1531.169312,-1484.254028,1508.881348,0.003365,0nubb
4,122,-1531.922852,-1491.922974,1513.565308,0.003935,0nubb
...,...,...,...,...,...,...
144,70151,-1226.355713,-2456.864014,3708.881348,0.001812,Bkg
145,70151,-1240.709229,-2467.480225,3718.807373,0.162111,Bkg
146,70151,-1254.262695,-2465.050537,3709.687256,0.059494,Bkg
147,70151,-1597.664429,-2015.367432,2807.327148,0.006204,Bkg


In [4]:
def MinMaxScale(df, label):
    # Min-Max scaling
    var_min = df[label].min()
    var_max = df[label].max()
    df[label] = (df[label] - var_min) / (var_max - var_min)
    return df

# Function for Min-Max normalization
def normalize_group(group):
    # Avoid division by zero if a group has only one point or max == min
    if group.max() == group.min():
        return group * 0.0
    return (group - group.min()) / (group.max() - group.min())

# Normalize the columns
xyz_mean = df[["x", "y", "z"]].mean()
xyz_std  = df[["x", "y", "z"]].std()
df[["x", "y", "z"]] = (df[["x", "y", "z"]] - xyz_mean) / xyz_std


# Apply clipping to the energy
df['energy'] = df['energy'].clip(upper=0.4)

df = MinMaxScale(df, "energy")

display(df)

,event_id,x,y,z,energy,Type
0,122,-1.098896,-1.057378,-0.649942,0.086234,0nubb
1,122,-1.090830,-1.057312,-0.647092,0.234576,0nubb
2,122,-1.087598,-1.050603,-0.646427,0.104714,0nubb
3,122,-1.082201,-1.053451,-0.649450,0.007603,0nubb
4,122,-1.082736,-1.058896,-0.646404,0.009030,0nubb
...,...,...,...,...,...,...
144,70151,-0.865725,-1.744071,0.780961,0.003718,Bkg
145,70151,-0.875918,-1.751610,0.787414,0.404792,Bkg
146,70151,-0.885544,-1.749884,0.781485,0.148040,Bkg
147,70151,-1.129425,-1.430578,0.194782,0.014707,Bkg


In [5]:
# Assume df has columns: event_id, x, y, z, energy, Type
# Type: "0nubb" or "Bkg"
df['label'] = (df['Type'] == "0nubb").astype(int)

# Event-level labels
event_labels = df.groupby('event_id')['label'].first()
event_ids    = event_labels.index.values
event_y      = event_labels.values

# Split 70/20/10
ev_tmp, ev_test, y_tmp, y_test   = train_test_split(event_ids, event_y, test_size=0.10, stratify=event_y, random_state=42)
ev_train, ev_val, y_train, y_val = train_test_split(ev_tmp, y_tmp, test_size=2/9, stratify=y_tmp, random_state=42)


In [6]:
def voxelize_event(event_df):
    
    VOXEL_SIZE = 8.0 # mm

    coords = event_df[['x','y','z']].values
    energy = event_df[['energy']].values

    coords = coords / VOXEL_SIZE
    coords = coords.astype(int)

    return coords, energy


def merge_duplicates(coords, feats):

    uniq, inv = np.unique(coords, axis=0, return_inverse=True)

    merged_feats = np.zeros((len(uniq),1))

    for i in range(len(feats)):
        merged_feats[inv[i]] += feats[i]

    return uniq, merged_feats

In [10]:
class EventDataset(Dataset):
    def __init__(self, events, labels):
        self.events = events
        self.labels = labels

    def __getitem__(self, idx):
        event = self.events[idx]
        coords, feats = voxelize_event(event)
        coords, feats = merge_duplicates(coords, feats)

        # batch_index added later in collate_fn
        feats = torch.FloatTensor(feats).view(-1, feats.shape[1])
        label = torch.LongTensor([self.labels[idx]])
        return coords, feats, label

    def __len__(self):
        return len(self.events)

class SparseClassifier(ME.MinkowskiNetwork):

    def __init__(self):
        super().__init__(D=3)

        self.conv1 = ME.MinkowskiConvolution(1, 16, kernel_size=3, stride=1, dimension=3)
        self.bn1   = ME.MinkowskiBatchNorm(16)
        self.relu1 = ME.MinkowskiReLU()

        self.conv2 = ME.MinkowskiConvolution(16, 32, kernel_size=3, stride=2, dimension=3)
        self.bn2   = ME.MinkowskiBatchNorm(32)
        self.relu2 = ME.MinkowskiReLU()

        self.conv3 = ME.MinkowskiConvolution(32, 64, kernel_size=3, stride=2, dimension=3)
        self.bn3   = ME.MinkowskiBatchNorm(64)
        self.relu3 = ME.MinkowskiReLU()

        self.pool  = ME.MinkowskiGlobalMaxPooling()
        self.fc    = ME.MinkowskiLinear(64, 2)

    def forward(self, coords, feats):

        x = ME.SparseTensor(feats, coordinates=coords)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.pool(x)
        x = self.fc(x)

        return x.F   # convert SparseTensor → dense tensor

In [18]:
def sparse_collate_fn(batch):
    coords, feats, labels = [], [], []
    for c, f, l in batch:
        coords.append(c)
        feats.append(f)
        labels.append(l)
    coords = ME.utils.batched_coordinates(coords)
    feats  = torch.cat(feats, dim=0)
    labels = torch.cat(labels)
    return coords, feats, labels

def make_event_list(df, event_ids):
    grouped = df.groupby('event_id')
    events = [grouped.get_group(eid).copy() for eid in event_ids]
    labels = [g['label'].iloc[0] for g in events]
    return events, labels

train_events, train_labels = make_event_list(df, ev_train)
val_events, val_labels     = make_event_list(df, ev_val)
test_events, test_labels   = make_event_list(df, ev_test)

train_dataset = EventDataset(train_events, train_labels)
val_dataset   = EventDataset(val_events,   val_labels)
test_dataset  = EventDataset(test_events,  test_labels)


BATCH_SIZE = 2000  # adjust based on GPU memory
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=sparse_collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=sparse_collate_fn)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=sparse_collate_fn)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SparseClassifier().cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

EPOCHS = 10  # adjust

for epoch in range(EPOCHS):

    # ---- train ----
    model.train()
    train_loss = 0
    for coords, feats, y in train_loader:
        coords, feats, y = coords.to(device), feats.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(coords, feats)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ---- validate ----
    model.eval()
    val_loss = 0
    correct  = 0
    total    = 0
    with torch.no_grad():
        for coords, feats, y in val_loader:
            coords, feats, y = coords.to(device), feats.to(device), y.to(device)
            logits = model(coords, feats)
            loss = criterion(logits, y)
            val_loss += loss.item()
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val Acc: {val_acc:.3f}")

# ================================
# 9. Test evaluation
# ================================

model.eval()
correct = 0
total   = 0
with torch.no_grad():
    for coords, feats, y in test_loader:
        coords, feats, y = coords.to(device), feats.to(device), y.to(device)
        logits = model(coords, feats)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total   += y.size(0)

print("Test Accuracy:", correct / total)

Epoch 1/10 | Train Loss: 16.481 | Val Loss: 4.849 | Val Acc: 0.514
Epoch 2/10 | Train Loss: 16.376 | Val Loss: 4.849 | Val Acc: 0.491
Epoch 3/10 | Train Loss: 16.380 | Val Loss: 4.902 | Val Acc: 0.480
Epoch 4/10 | Train Loss: 16.361 | Val Loss: 5.395 | Val Acc: 0.487
Epoch 5/10 | Train Loss: 16.350 | Val Loss: 4.950 | Val Acc: 0.487
Epoch 6/10 | Train Loss: 16.366 | Val Loss: 4.979 | Val Acc: 0.480
Epoch 7/10 | Train Loss: 16.352 | Val Loss: 5.279 | Val Acc: 0.483
Epoch 8/10 | Train Loss: 16.355 | Val Loss: 4.881 | Val Acc: 0.487
Epoch 9/10 | Train Loss: 16.355 | Val Loss: 5.085 | Val Acc: 0.487
Epoch 10/10 | Train Loss: 16.351 | Val Loss: 4.920 | Val Acc: 0.479
Test Accuracy: 0.475611542405156
